# Import Library

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('PM25_train.csv')
data.head()

,device_id,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon
0,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
1,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
2,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
3,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755
4,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755


# EDA

In [3]:
data["PM2.5"].describe()


count    1.116554e+06
mean     3.818598e+01
std      1.747436e+01
min      0.000000e+00
25%      2.700000e+01
50%      3.800000e+01
75%      4.800000e+01
max      7.860000e+02
Name: PM2.5, dtype: float64

In [4]:

#plt.plot(X['Date'],X['PM2.5'],'.')
#plt.plot(data['lat'],data['lon'],'.')

In [17]:
# 
TS_WEIGHT = 1000000000
#data["Timestamp"] = data["Date"].astype(str) + " " + data["Time"]
data["Timestamp"] =pd.to_datetime(data["Date"].astype(str) + " " + data["Time"], format='%Y-%m-%d %H:%M:%S').astype(np.int64) 
data["Timestamp"] = data[ "Timestamp"] // TS_WEIGHT
#"2017-01-01 08:03:09" => 1483257789.0
#"2017-01-01 08:09:04" => 1483258144.0
data.head()

,device_id,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp
0,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755,1483257789
1,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755,1483257789
2,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755,1483257789
3,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755,1483258144
4,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755,1483258144


# X,y


In [18]:

X = data.drop(['device_id','Date','Time','PM2.5'],axis=1)
X.head()

,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp
0,0.0,0.0,24.12,83.0,23.741,120.755,1483257789
1,0.0,0.0,24.12,83.0,23.741,120.755,1483257789
2,0.0,0.0,24.12,83.0,23.741,120.755,1483257789
3,0.0,0.0,24.12,82.0,23.741,120.755,1483258144
4,0.0,0.0,24.12,82.0,23.741,120.755,1483258144


In [7]:
y = data["PM2.5"].as_matrix()
y.shape

(1116554,)

# model

In [20]:
from sklearn import linear_model

linear = linear_model.LinearRegression(normalize=True) #15.884043066/2.90485

ard = linear_model.ARDRegression()

alpha =  1e-4

ridge = linear_model.Ridge(alpha = alpha, normalize=True) # 255

#lasso
lasso = linear_model.Lasso(alpha = alpha, normalize=True) #17
lars = linear_model.LassoLars() #305
#linear_model.LarsCV() #255.522

reg = linear

reg.fit (X,y)
print (reg.coef_)

[  3.50697950e-01   3.60608277e-01   3.96171487e-02   5.18417511e-02
  -8.49432652e+00   3.25402864e+00  -1.62112822e-07]


In [21]:
y_pred = reg.predict(X)
mse = mean_squared_error(y_pred,y)
print(mse)

15.8840430666


In [10]:
# predict

In [11]:
test_data = pd.read_csv("PM25_test.csv")
test_data["Timestamp"] =pd.to_datetime(test_data["Date"].astype(str) + " " + test_data["Time"], format='%Y-%m-%d %H:%M:%S').astype(np.int64) 
test_data["Timestamp"] = test_data[ "Timestamp"] // TS_WEIGHT 
test_x = test_data
test_x = test_x.drop(['device_id','Date','Time'],axis=1)
test_x.head()

,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp
0,0.0,0.0,18.62,92.0,23.741,120.755,1485821045
1,0.0,0.0,18.75,92.0,23.741,120.755,1485821401
2,0.0,0.0,18.62,92.0,23.741,120.755,1485821757
3,0.0,0.0,18.62,92.0,23.741,120.755,1485822113
4,0.0,0.0,18.62,92.0,23.741,120.755,1485822468


In [12]:
#predict
pre = reg.predict(test_x.values)
pre

array([ 7.07258296,  7.07767548,  7.07246754, ...,  9.25719863,
        7.8293737 ,  8.23694387])

In [13]:
result = test_data[["device_id"]]
result["pred_pm25"] = pre
result.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,device_id,pred_pm25
0,28C2DDDD415C,7.072583
1,28C2DDDD415C,7.077675
2,28C2DDDD415C,7.072468
3,28C2DDDD415C,7.072410
4,28C2DDDD415C,7.072352


In [14]:
ids = result.groupby(['device_id'])


In [15]:
#unique device id

pd_mean_pm25 = result.groupby(['device_id'])['pred_pm25'].mean().reset_index()
pd_mean_pm25.head()

,device_id,pred_pm25
0,28C2DDDD415C,6.934871
1,28C2DDDD47BC,2.506886
2,74DA3895C1F8,13.371260
3,74DA3895C1FA,11.435568
4,74DA3895C200,13.012615


In [16]:
#np.savetxt(, result.tolist(), delimiter = ',')
pd_mean_pm25.to_csv('l1-sumbmission.csv', index  = False)